<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/text_embedding_limitations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet neo4j langchain-community langchain-openai

In [20]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
import getpass

In [21]:
os.environ["NEO4J_URI"] = "bolt://44.204.178.84:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "minimums-triangle-saving"

graph = Neo4jGraph(refresh_schema=False)

In [22]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [13]:
graph.query("""
MATCH (m:Movie)
WHERE m.imdbRating IS NOT NULL
WITH m
ORDER BY m.imdbRating DESC
LIMIT 1000
SET m:Target
""")

[]

In [18]:
graph.query("""MATCH (m:Target) RETURN m LIMIT 1""")

[{'m': {'languages': ['English'],
   'year': 1995,
   'imdbId': '0114709',
   'runtime': 81,
   'imdbRating': 8.3,
   'movieId': '1',
   'countries': ['USA'],
   'imdbVotes': 591836,
   'title': 'Toy Story',
   'url': 'https://themoviedb.org/movie/862',
   'revenue': 373554033,
   'tmdbId': '862',
   'plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.",
   'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg',
   'released': '1995-11-22',
   'budget': 30000000}}]

In [23]:
neo4j_vector = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    index_name="movies",
    node_label="Target",
    text_node_properties=["plot", "title", "year", "imdbRating"],
    embedding_node_property="embedding",
)

In [25]:
neo4j_vector.similarity_search("What is a movie where a little boy meets his hero?")

[Document(metadata={'budget': 70000000, 'movieId': '2761', 'tmdbId': '10386', 'imdbVotes': 123948, 'runtime': 86, 'countries': ['USA'], 'imdbId': '0129167', 'url': 'https://themoviedb.org/movie/10386', 'released': '1999-08-06', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/dUFiLEEuTZyUiQrN6oYMbMtDaXD.jpg', 'revenue': 23159305}, page_content='\nplot: A young boy befriends a giant robot from outer space that a paranoid government agent wants to destroy.\ntitle: Iron Giant, The\nyear: 1999\nimdbRating: 8.0'),
 Document(metadata={'budget': 8000000, 'movieId': '1259', 'tmdbId': '235', 'imdbVotes': 261725, 'runtime': 89, 'countries': ['USA'], 'imdbId': '0092005', 'url': 'https://themoviedb.org/movie/235', 'released': '1986-08-22', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/vz0w9BSehcqjDcJOjRaCk7fgJe7.jpg', 'revenue': 52287414}, page_content='\nplot: After the death of a friend, a writer recounts a boyhood journey to

In [26]:
neo4j_vector.similarity_search("Which movies are from year 2016?")

[Document(metadata={'budget': 4000000, 'movieId': '116897', 'tmdbId': '265195', 'imdbVotes': 76453, 'runtime': 122, 'countries': ['Argentina', ' Spain'], 'imdbId': '3011894', 'url': 'https://themoviedb.org/movie/265195', 'released': '2014-08-21', 'languages': ['Spanish'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/pvSty5MCAxdj8LlAEXCDfM1YNOl.jpg', 'revenue': 27007072}, page_content='\nplot: Six short stories that explore the extremities of human behavior involving people in distress.\ntitle: Wild Tales\nyear: 2014\nimdbRating: 8.1'),
 Document(metadata={'budget': 120000000, 'movieId': '97938', 'tmdbId': '87827', 'imdbVotes': 420635, 'runtime': 127, 'countries': ['USA', ' Taiwan', ' UK'], 'imdbId': '0454876', 'url': 'https://themoviedb.org/movie/87827', 'released': '2012-11-21', 'languages': ['English', ' Tamil', ' French', ' Japanese', ' Hindi', ' Chinese'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/mYDKm9HxImm8PRru3KbkHAe1cmk.jpg', 'revenue': 609016565}, p

In [30]:
neo4j_hybrid = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    index_name="movies",
    node_label="Target",
    text_node_properties=["plot", "title", "year", "imdbRating"],
    embedding_node_property="embedding",
    search_type='hybrid'
)

neo4j_hybrid.similarity_search("Which movies are from year 2016?")

[Document(metadata={'budget': 4000000, 'tmdbId': '265195', 'movieId': '116897', 'imdbVotes': 76453, 'runtime': 122, 'countries': ['Argentina', ' Spain'], 'imdbId': '3011894', 'url': 'https://themoviedb.org/movie/265195', 'released': '2014-08-21', 'languages': ['Spanish'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/pvSty5MCAxdj8LlAEXCDfM1YNOl.jpg', 'revenue': 27007072}, page_content='\nplot: Six short stories that explore the extremities of human behavior involving people in distress.\ntitle: Wild Tales\nyear: 2014\nimdbRating: 8.1'),
 Document(metadata={'tmdbId': '114524', 'movieId': '26655', 'imdbVotes': 492, 'runtime': 79, 'countries': ['USA'], 'imdbId': '0097099', 'url': 'https://themoviedb.org/movie/114524', 'released': '1990-10-26', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/tAmvWQl1R0uiydLnG0o9PgRyanS.jpg'}, page_content='\nplot: A collection of profiles of people dead from AIDS who are remembered in the AIDS Memorial Quilt.\n

In [33]:
graph.query("""MATCH (m:Target) WHERE m.year = 2016 RETURN m.title AS title""")

[{'title': 'Deadpool'},
 {'title': 'Zootopia'},
 {'title': 'Sing Street'},
 {'title': 'Author: The JT LeRoy Story'}]

In [27]:
neo4j_vector.similarity_search("Which movie has the highest imdb score?")

[Document(metadata={'budget': 2540800, 'movieId': '899', 'tmdbId': '872', 'imdbVotes': 144026, 'runtime': 103, 'countries': ['USA'], 'imdbId': '0045152', 'url': 'https://themoviedb.org/movie/872', 'released': '1952-04-11', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/w03EiJVHP8Un77boQeE7hg9DVdU.jpg', 'revenue': 7200000}, page_content="\nplot: A silent film production company and cast make a difficult transition to sound.\ntitle: Singin' in the Rain\nyear: 1952\nimdbRating: 8.3"),
 Document(metadata={'movieId': '7979', 'tmdbId': '42632', 'imdbVotes': 2179, 'runtime': 78, 'countries': ['USA'], 'imdbId': '0064689', 'url': 'https://themoviedb.org/movie/42632', 'released': '1969-06-02', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/wEB3c5Y5spAHOQaFWcsfFky3SWb.jpg'}, page_content='\nplot: A film about the greatest pre-Woodstock rock music festival.\ntitle: Monterey Pop\nyear: 1968\nimdbRating: 8.1'),
 Document(metadat

In [34]:
graph.query("""MATCH (m:Target) RETURN m.title AS title, m.imdbRating AS rating ORDER BY rating DESC LIMIT 4""")

[{'title': 'Band of Brothers', 'rating': 9.6},
 {'title': 'Civil War, The', 'rating': 9.5},
 {'title': 'Shawshank Redemption, The', 'rating': 9.3},
 {'title': 'Cosmos', 'rating': 9.3}]

In [36]:
neo4j_vector.similarity_search("How many movies are from the year 2015?")

[Document(metadata={'budget': 245000000, 'movieId': '122886', 'tmdbId': '140607', 'imdbVotes': 457987, 'runtime': 135, 'countries': ['USA'], 'imdbId': '2488496', 'url': 'https://themoviedb.org/movie/140607', 'released': '2015-12-18', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg', 'revenue': 2068223624}, page_content='\nplot: Three decades after the defeat of the Galactic Empire, a new threat arises. The First Order attempts to rule the galaxy and only a ragtag group of heroes can stop them, along with the help of the Resistance.\ntitle: Star Wars: Episode VII - The Force Awakens\nyear: 2015\nimdbRating: 8.4'),
 Document(metadata={'budget': 4000000, 'movieId': '27800', 'tmdbId': '11049', 'imdbVotes': 10885, 'runtime': 68, 'countries': ['Japan', ' France'], 'imdbId': '0368667', 'url': 'https://themoviedb.org/movie/11049', 'released': '2003-05-28', 'languages': ['English'], 'poster': 'https://image.tmdb.org/t/p/w440_and

In [37]:
graph.query("""MATCH (m:Target) WHERE m.year = 2015 RETURN count(*)""")

[{'count(*)': 20}]